In [72]:
import pandas as pd

In [80]:
hosts = pd.read_csv('../data/olympic_hosts.csv')
hosts = hosts[hosts['game_season'] == 'Summer']
hosts.drop(columns=['game_slug', 'game_end_date',
           'game_start_date', 'game_name', 'game_season'], inplace=True)

athletes = pd.read_csv('../data/athletes.csv')
athletes.drop(columns=['id', 'died'], inplace=True)
athletes['height'] = athletes['height'].str.replace(' cm', '')
athletes['weight'] = athletes['weight'].str.replace(' kg', '')
athletes['weight'] = athletes['weight'].str.extract(r'(\d{2})')
athletes = athletes[athletes['game'].str.contains('Summer')]

athletes['medal'] = athletes['medal'].fillna("None")

athletes['game_year'] = athletes['game'].str.extract(
    r'(\d{4})').astype(int)
athletes.dropna(inplace=True)

athletes['game_year'] = athletes['game_year'].astype(int)
athletes['height'] = athletes['height'].astype(int)
athletes['weight'] = athletes['weight'].astype(int)
athletes['born_year'] = athletes['born'].str.extract(
    r'(\d{4})').astype(int)
athletes['age'] = athletes['game_year'] - athletes['born_year']
athletes['age'] = athletes['age'].astype(int)

athletes = pd.merge(athletes, hosts, on='game_year', how='left')
athletes['is_host'] = athletes['team'] == athletes['game_location']

athletes.drop(columns=['born', 'team', 'game', 'sport',
              'event', 'born_year', 'game_location'], inplace=True)

athletes

,name,gender,height,weight,noc,medal,game_year,age,is_host
0,Meryem Erdoğan,Female,172,55,TUR,None,2016,26,False
1,Meryem Erdoğan,Female,172,55,TUR,None,2020,30,False
2,Meryem Erdoğan,Female,172,55,TUR,None,2020,30,False
3,Maurice Maina,Male,158,47,KEN,None,1988,25,False
4,Maurice Maina,Male,158,47,KEN,None,1988,25,False
...,...,...,...,...,...,...,...,...,...
271466,Caitlin Bilodeaux-Banos,Female,170,64,USA,None,1988,23,False
271467,Caitlin Bilodeaux-Banos,Female,170,64,USA,None,1988,23,False
271468,Caitlin Bilodeaux-Banos,Female,170,64,USA,None,1992,27,False
271469,Caitlin Bilodeaux-Banos,Female,170,64,USA,None,1992,27,False


In [58]:
athletes[athletes['name'] == 'Aaron Brown']

,name,gender,height,weight,noc,medal,game_year,age,is_host
4425,Aaron Brown,Male,198,79,CAN,Bronze,2016,24,False
4426,Aaron Brown,Male,198,79,CAN,Silver,2020,28,False
4427,Aaron Brown,Male,198,79,CAN,Silver,2020,28,False


In [59]:
paris_data = pd.read_csv('../data/team_canada_paris_2024.csv')
current_athletes = paris_data.rename(columns={'SPORT (EN)': 'Sport',
                                              'FIRST NAME / PRÉNOM': 'First Name',
                                              'LAST NAME / NOM': 'Last Name',
                                              'AGE (AT GAMES) / ÂGE (AUX JEUX)': 'Age'})

# Reformat names
current_athletes['Name'] = current_athletes['First Name'] + \
    ' ' + current_athletes['Last Name']
current_athletes = current_athletes.drop(['First Name', 'Last Name', 'PRONUNCIATION / PRONONCIATION',
                                         'SPORT (FR)', 'HOMETOWN / LIEU DE RÉSIDENCE', 'HOME PROVINCE / PROVINCE DE RÉSIDENCE', 'LANGUAGES / LANGUES'], axis=1)
current_athletes = current_athletes.merge(
    athletes, left_on=['Name'], right_on=['name'], how='inner')
current_athletes['game_year'] = 2024
current_athletes.drop(columns=['Sport', 'age', 'medal'], inplace=True)
current_athletes.rename(columns={'Age': 'age'}, inplace=True)
current_athletes

,age,Name,name,gender,height,weight,noc,game_year,is_host
0,32.0,Aaron Brown,Aaron Brown,Male,198,79,CAN,2024,False
1,32.0,Aaron Brown,Aaron Brown,Male,198,79,CAN,2024,False
2,32.0,Aaron Brown,Aaron Brown,Male,198,79,CAN,2024,False
3,29.0,Andre De Grasse,Andre De Grasse,Male,176,70,CAN,2024,False
4,29.0,Andre De Grasse,Andre De Grasse,Male,176,70,CAN,2024,False
...,...,...,...,...,...,...,...,...,...
62,24.0,Taylor Ruck,Taylor Ruck,Female,180,60,CAN,2024,False
63,24.0,Taylor Ruck,Taylor Ruck,Female,180,60,CAN,2024,False
64,24.0,Taylor Ruck,Taylor Ruck,Female,180,60,CAN,2024,False
65,24.0,Taylor Ruck,Taylor Ruck,Female,180,60,CAN,2024,False


In [60]:
economicData = pd.read_csv('../out/economic_data.csv')
economicData

,Series,1960,1961,1962,1963,1964,1965,1966,1967,1968,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,noc
0,NY.GDP.MKTP.KD,-2.155502e+09,-1.913017e+09,-1.670532e+09,-1.428047e+09,-1.185563e+09,-9.430780e+08,-7.005932e+08,-4.581085e+08,-2.156238e+08,...,1.176433e+10,1.221168e+10,1.270250e+10,1.296770e+10,1.253949e+10,1.365658e+10,1.431979e+10,1.481227e+10,1.336352e+10,ALB
1,NY.GDP.MKTP.KD.ZG,-3.260157e-01,-2.469403e-01,-1.678649e-01,-8.878944e-02,-9.714020e-03,6.936140e-02,1.484368e-01,2.275122e-01,3.065877e-01,...,3.314981e+00,3.802599e+00,4.019346e+00,2.087712e+00,-3.302082e+00,8.908528e+00,4.856402e+00,3.439149e+00,4.734811e+00,ALB
2,EG.ELC.ACCS.ZS,9.974298e+01,9.974442e+01,9.974586e+01,9.974729e+01,9.974873e+01,9.975017e+01,9.975160e+01,9.975304e+01,9.975448e+01,...,9.990000e+01,9.990000e+01,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02,9.983352e+01,9.983496e+01,ALB
3,EG.CFT.ACCS.ZS,-5.201789e+01,-4.971937e+01,-4.742085e+01,-4.512233e+01,-4.282381e+01,-4.052530e+01,-3.822678e+01,-3.592826e+01,-3.362974e+01,...,7.830000e+01,7.960000e+01,8.070000e+01,8.200000e+01,8.310000e+01,8.360000e+01,8.460000e+01,9.278874e+01,9.508725e+01,ALB
4,SP.POP.GROW,3.194951e+00,3.120855e+00,3.056731e+00,2.953749e+00,2.880686e+00,2.754021e+00,2.634564e+00,2.630190e+00,2.842511e+00,...,-1.598804e-01,-9.197229e-02,-2.467320e-01,-4.260074e-01,-5.742070e-01,-9.269181e-01,-1.215790e+00,-1.148418e+00,-1.497661e+00,ALB
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,NY.GDP.PCAP.KD.ZG,1.484089e+00,3.076210e+00,-1.666989e+00,2.985427e+00,-4.143906e+00,1.682986e+00,-1.604962e+00,5.021605e+00,-1.194617e+00,...,-1.177893e+00,1.974846e+00,2.909395e+00,-8.177320e+00,-9.670405e+00,6.271613e+00,4.387997e+00,2.783839e+00,-3.614132e-01,ZIM
1496,NY.GDP.DEFL.KD.ZG,-6.363735e+01,-2.041300e+00,4.696178e-01,-2.347740e+00,6.144012e+00,2.704143e+00,-3.729967e+00,5.765617e-01,3.866207e+00,...,2.014095e+00,3.056905e+00,2.007696e+02,2.253946e+02,6.049459e+02,1.132950e+02,2.645806e+02,8.604988e+02,1.418955e+02,ZIM
1497,NY.ADJ.NNTY.KD,-2.867420e+10,-2.788785e+10,-2.710150e+10,-2.631515e+10,-2.552880e+10,-2.474245e+10,-2.395610e+10,-2.316976e+10,-2.238341e+10,...,1.520712e+10,1.590361e+10,1.994242e+10,1.841870e+10,1.662388e+10,1.775851e+10,2.007946e+10,2.086581e+10,2.165216e+10,ZIM
1498,SH.XPD.CHEX.GD.ZS,4.006639e+01,3.945806e+01,3.884972e+01,3.824139e+01,3.763305e+01,3.702472e+01,3.641638e+01,3.580805e+01,3.519971e+01,...,7.466447e+00,6.363583e+00,4.670392e+00,3.232802e+00,2.954401e+00,2.789751e+00,2.349618e+00,1.741283e+00,1.132947e+00,ZIM


In [61]:
economic_data_melted = pd.melt(
    economicData, id_vars=['Series', 'noc'], var_name='year', value_name='value')
economic_data_melted['year'] = economic_data_melted['year'].astype(int)
economic_data_melted

,Series,noc,year,value
0,NY.GDP.MKTP.KD,ALB,1960,-2.155502e+09
1,NY.GDP.MKTP.KD.ZG,ALB,1960,-3.260157e-01
2,EG.ELC.ACCS.ZS,ALB,1960,9.974298e+01
3,EG.CFT.ACCS.ZS,ALB,1960,-5.201789e+01
4,SP.POP.GROW,ALB,1960,3.194951e+00
...,...,...,...,...
97495,NY.GDP.PCAP.KD.ZG,ZIM,2024,-3.614132e-01
97496,NY.GDP.DEFL.KD.ZG,ZIM,2024,1.418955e+02
97497,NY.ADJ.NNTY.KD,ZIM,2024,2.165216e+10
97498,SH.XPD.CHEX.GD.ZS,ZIM,2024,1.132947e+00


In [62]:
merged_curr_data = pd.merge(current_athletes, economic_data_melted, left_on=[
    'noc', 'game_year'], right_on=['noc', 'year'], how='left')
merged_curr_data.dropna(inplace=True)
merged_curr_data

,age,Name,name,gender,height,weight,noc,game_year,is_host,Series,year,value
0,32.0,Aaron Brown,Aaron Brown,Male,198,79,CAN,2024,False,NY.GDP.MKTP.KD,2024,1.743957e+12
1,32.0,Aaron Brown,Aaron Brown,Male,198,79,CAN,2024,False,NY.GDP.MKTP.KD.ZG,2024,1.115198e+00
2,32.0,Aaron Brown,Aaron Brown,Male,198,79,CAN,2024,False,EG.ELC.ACCS.ZS,2024,1.000000e+02
3,32.0,Aaron Brown,Aaron Brown,Male,198,79,CAN,2024,False,EG.CFT.ACCS.ZS,2024,1.000000e+02
4,32.0,Aaron Brown,Aaron Brown,Male,198,79,CAN,2024,False,SP.POP.GROW,2024,9.998359e-01
...,...,...,...,...,...,...,...,...,...,...,...,...
1000,24.0,Taylor Ruck,Taylor Ruck,Female,180,60,CAN,2024,False,NY.GDP.PCAP.KD.ZG,2024,1.189872e-01
1001,24.0,Taylor Ruck,Taylor Ruck,Female,180,60,CAN,2024,False,NY.GDP.DEFL.KD.ZG,2024,1.598778e+00
1002,24.0,Taylor Ruck,Taylor Ruck,Female,180,60,CAN,2024,False,NY.ADJ.NNTY.KD,2024,1.440632e+12
1003,24.0,Taylor Ruck,Taylor Ruck,Female,180,60,CAN,2024,False,SH.XPD.CHEX.GD.ZS,2024,1.230489e+01


In [63]:
merged_data = pd.merge(athletes, economic_data_melted, left_on=[
                       'noc', 'game_year'], right_on=['noc', 'year'], how='left')
merged_data.dropna(inplace=True)
merged_data

,name,gender,height,weight,noc,medal,game_year,age,is_host,Series,year,value
4,Kerstin Müller,Female,179,70,GER,Gold,1992,23,False,NY.GDP.MKTP.KD,1992.0,2.509124e+12
5,Kerstin Müller,Female,179,70,GER,Gold,1992,23,False,NY.GDP.MKTP.KD.ZG,1992.0,1.923077e+00
6,Kerstin Müller,Female,179,70,GER,Gold,1992,23,False,EG.ELC.ACCS.ZS,1992.0,1.000000e+02
7,Kerstin Müller,Female,179,70,GER,Gold,1992,23,False,EG.CFT.ACCS.ZS,1992.0,1.000000e+02
8,Kerstin Müller,Female,179,70,GER,Gold,1992,23,False,SP.POP.GROW,1992.0,7.603469e-01
...,...,...,...,...,...,...,...,...,...,...,...,...
355651,Aleksa Šaponjić,Male,191,98,SRB,Bronze,2012,20,False,NY.GDP.PCAP.KD.ZG,2012.0,-1.983789e-01
355652,Aleksa Šaponjić,Male,191,98,SRB,Bronze,2012,20,False,NY.GDP.DEFL.KD.ZG,2012.0,6.199339e+00
355653,Aleksa Šaponjić,Male,191,98,SRB,Bronze,2012,20,False,NY.ADJ.NNTY.KD,2012.0,3.064734e+10
355654,Aleksa Šaponjić,Male,191,98,SRB,Bronze,2012,20,False,SH.XPD.CHEX.GD.ZS,2012.0,9.328204e+00


In [64]:
final_data = merged_data.pivot_table(index=['name', 'gender', 'height', 'weight', 'noc', 'medal', 'game_year', 'age', 'is_host'],
                                     columns='Series',
                                     values='value').reset_index()
final_data.columns.name = None
final_data

,name,gender,height,weight,noc,medal,game_year,age,is_host,AG.LND.TOTL.K2,...,NY.GDP.DEFL.KD.ZG,NY.GDP.MKTP.KD,NY.GDP.MKTP.KD.ZG,NY.GDP.PCAP.KD,NY.GDP.PCAP.KD.ZG,SH.XPD.CHEX.GD.ZS,SL.UEM.TOTL.ZS,SN.ITK.MSFI.ZS,SP.POP.GROW,SP.POP.TOTL
0,A'ja Wilson,Female,193,88,USA,Gold,2020,24,False,9147420.0,...,1.320618,1.969097e+13,-2.213469,59394.778327,-3.156444,18.756220,8.055000,8.000000,0.969000,331526933.0
1,A. J. Hinch,Male,185,91,USA,Bronze,1996,22,True,9158960.0,...,1.830765,1.152570e+13,3.772773,42783.816570,2.572464,12.822659,5.451000,18.542857,1.163412,269394000.0
2,Aage Birch,Male,172,70,DEN,Silver,1968,42,False,40000.0,...,5.632591,1.226516e+11,5.553077,25211.629553,4.912388,5.929960,8.484263,2.621429,0.608832,4864883.0
3,Aaron Brown,Male,198,79,CAN,Bronze,2016,24,False,8965590.0,...,0.716978,1.572674e+12,1.038551,43551.342602,-0.098296,11.031925,7.038000,3.980000,1.131540,36110803.0
4,Aaron Brown,Male,198,79,CAN,Silver,2020,28,False,8965590.0,...,1.070810,1.611126e+12,-5.038233,42366.128519,-6.062406,13.035118,9.657000,6.500000,1.084369,38028638.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14623,Ștefan Rusu,Male,178,74,ROU,Bronze,1984,28,False,230340.0,...,150.728782,5.063194e+10,-1.552431,1440.036280,-0.937574,3.803981,8.460102,35.200000,0.354280,22640547.0
14624,Ștefan Rusu,Male,178,74,ROU,Gold,1980,24,False,230340.0,...,169.783663,3.317131e+10,-2.222148,409.694265,-1.607578,3.596211,8.786418,37.514286,0.527314,22207282.0
14625,Ștefan Rusu,Male,178,74,ROU,Silver,1976,20,False,230340.0,...,188.838544,1.571068e+10,-2.891865,-620.647750,-2.277583,3.388442,9.112733,39.828571,1.204588,21551634.0
14626,Ștefan Tașnadi,Male,183,11,ROU,Silver,1984,31,False,230340.0,...,150.728782,5.063194e+10,-1.552431,1440.036280,-0.937574,3.803981,8.460102,35.200000,0.354280,22640547.0


In [65]:
final_curr_data = merged_curr_data.pivot_table(index=['name', 'gender', 'height', 'weight', 'noc', 'game_year', 'age', 'is_host'],
                                               columns='Series',
                                               values='value').reset_index()
final_curr_data.columns.name = None
final_curr_data

,name,gender,height,weight,noc,game_year,age,is_host,AG.LND.TOTL.K2,EG.CFT.ACCS.ZS,...,NY.GDP.DEFL.KD.ZG,NY.GDP.MKTP.KD,NY.GDP.MKTP.KD.ZG,NY.GDP.PCAP.KD,NY.GDP.PCAP.KD.ZG,SH.XPD.CHEX.GD.ZS,SL.UEM.TOTL.ZS,SN.ITK.MSFI.ZS,SP.POP.GROW,SP.POP.TOTL
0,Aaron Brown,Male,198,79,CAN,2024,32.0,False,8.953429e+06,100.0,...,1.598778,1.743957e+12,1.115198,47979.406403,0.118987,12.304889,5.709578,9.420000,0.999836,3.880394e+07
1,Andre De Grasse,Male,176,70,CAN,2024,29.0,False,8.953429e+06,100.0,...,1.598778,1.743957e+12,1.115198,47979.406403,0.118987,12.304889,5.709578,9.420000,0.999836,3.880394e+07
2,Ashley Lawrence,Female,163,59,CAN,2024,29.0,False,8.953429e+06,100.0,...,1.598778,1.743957e+12,1.115198,47979.406403,0.118987,12.304889,5.709578,9.420000,0.999836,3.880394e+07
3,Brendon Rodney,Male,195,80,CAN,2024,32.0,False,8.953429e+06,100.0,...,1.598778,1.743957e+12,1.115198,47979.406403,0.118987,12.304889,5.709578,9.420000,0.999836,3.880394e+07
4,Brian Yang,Male,180,65,CAN,2024,22.0,False,8.953429e+06,100.0,...,1.598778,1.743957e+12,1.115198,47979.406403,0.118987,12.304889,5.709578,9.420000,0.999836,3.880394e+07
5,Caileigh Filmer,Female,169,76,CAN,2024,27.0,False,8.953429e+06,100.0,...,1.598778,1.743957e+12,1.115198,47979.406403,0.118987,12.304889,5.709578,9.420000,0.999836,3.880394e+07
6,Catherine Beauchemin-Pinard,Female,161,57,CAN,2024,30.0,False,8.953429e+06,100.0,...,1.598778,1.743957e+12,1.115198,47979.406403,0.118987,12.304889,5.709578,9.420000,0.999836,3.880394e+07
7,Charity Williams,Female,162,68,CAN,2024,27.0,False,8.953429e+06,100.0,...,1.598778,1.743957e+12,1.115198,47979.406403,0.118987,12.304889,5.709578,9.420000,0.999836,3.880394e+07
8,Damian Warner,Male,185,83,CAN,2024,34.0,False,8.953429e+06,100.0,...,1.598778,1.743957e+12,1.115198,47979.406403,0.118987,12.304889,5.709578,9.420000,0.999836,3.880394e+07
9,Evan Dunfee,Male,182,70,CAN,2024,33.0,False,8.953429e+06,100.0,...,1.598778,1.743957e+12,1.115198,47979.406403,0.118987,12.304889,5.709578,9.420000,0.999836,3.880394e+07
